In [1]:
import os
os.chdir(r'C:\Users\dell\PycharmProjects\MachineLearning\Pandas\datasets\digit-recognizer')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns;sns.set()
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

X_train = train.iloc[:,1:] 
y_train = train.iloc[:,0]

X_test = test


In [3]:
X_train = X_train / 250.
X_test = X_test / 250.

In [14]:
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size = .8, test_size = .2, random_state = 5)

In [6]:
X_train = X_train.values.reshape((len(X_train), 28, 28, 1))
X_valid = X_valid.values.reshape((len(X_valid), 28 ,28,1))


In [9]:
X_test = X_test.values.reshape((len(X_test), 28, 28, 1))


In [11]:
model = Sequential([
    Conv2D(32, kernel_size = 5, activation = 'relu', input_shape = (28, 28,1)),
    MaxPool2D(pool_size = 2, strides = 2),
    Conv2D(64, kernel_size = 5, activation = 'relu'),
    MaxPool2D(pool_size = 2, strides = 2),
    Conv2D(128, kernel_size = 4, activation = 'relu'),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dropout(.5),
    Dense(64, activation = 'relu'),
    Dense(10, activation = 'softmax')
])
model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])

In [13]:
early_cb = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs =  30, batch_size=32, callbacks=[early_cb])

Train on 33600 samples, validate on 8400 samples
Epoch 1/30
33600/33600 [==============================] - 40s 1ms/step - loss: 0.2772 - accuracy: 0.9127 - val_loss: 0.0661 - val_accuracy: 0.9786
Epoch 2/30
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0719 - accuracy: 0.9809 - val_loss: 0.0751 - val_accuracy: 0.9795
Epoch 3/30
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0527 - accuracy: 0.9856 - val_loss: 0.0599 - val_accuracy: 0.9850
Epoch 4/30
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0400 - accuracy: 0.9889 - val_loss: 0.0463 - val_accuracy: 0.9881
Epoch 5/30
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0334 - accuracy: 0.9908 - val_loss: 0.0670 - val_accuracy: 0.9857
Epoch 6/30
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0278 - accuracy: 0.9924 - val_loss: 0.0649 - val_accuracy: 0.9857
Epoch 7/30
33600/33600 [==============================] - 39s 1ms/ste

In [16]:
checkpoint_cb = ModelCheckpoint('digit_model.h5', save_best_only=True)
early_cb = EarlyStopping(patience=10, restore_best_weights=True)

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs = 10, batch_size=32, callbacks = [checkpoint_cb, early_cb])

Train on 33600 samples, validate on 8400 samples
Epoch 1/10
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0405 - accuracy: 0.9888 - val_loss: 0.0583 - val_accuracy: 0.9852
Epoch 2/10
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0279 - accuracy: 0.9925 - val_loss: 0.0664 - val_accuracy: 0.9855
Epoch 3/10
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0222 - accuracy: 0.9937 - val_loss: 0.0639 - val_accuracy: 0.9861
Epoch 4/10
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0201 - accuracy: 0.9942 - val_loss: 0.0534 - val_accuracy: 0.9887
Epoch 5/10
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0151 - accuracy: 0.9958 - val_loss: 0.0598 - val_accuracy: 0.9882
Epoch 6/10
33600/33600 [==============================] - 39s 1ms/step - loss: 0.0151 - accuracy: 0.9957 - val_loss: 0.0614 - val_accuracy: 0.9885
Epoch 7/10
33600/33600 [==============================] - 39s 1ms/ste

# Submissions

In [22]:
ans = pd.DataFrame({
    'ImageId': 1 + np.arange(len(X_test)),
    'Label': model.predict_classes(X_test)
})
ans.to_csv('digit_prediction_custom1.csv', index = False)

# Data Augmentation

In [26]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=10, width_shift_range=.1, height_shift_range=.1, zoom_range=.1)
model_augment =  Sequential([
    Conv2D(32, kernel_size = 5, activation = 'relu', input_shape = (28, 28,1)),
    MaxPool2D(pool_size = 2, strides = 2),
    Conv2D(64, kernel_size = 5, activation = 'relu'),
    MaxPool2D(pool_size = 2, strides = 2),
    Conv2D(128, kernel_size = 4, activation = 'relu'),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dropout(.5),
    Dense(64, activation = 'relu'),
    Dense(10, activation = 'softmax')
])
model_augment.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])

In [29]:
augment_checkpoint_cb = ModelCheckpoint('digit_augmentation_model.h5', save_best_only=True)
augment_early_cb = EarlyStopping(patience=10, restore_best_weights= True)

In [32]:
model.fit_generator(gen.flow(X_train, y_train, batch_size = 32), validation_data=(X_valid, y_valid), steps_per_epoch=len(X_train) / 32, epochs = 30, callbacks=[augment_checkpoint_cb, augment_early_cb])

Epoch 1/30
1050/1050 [==============================] - 41s 39ms/step - loss: 0.0863 - accuracy: 0.9754 - val_loss: 0.0462 - val_accuracy: 0.9883
Epoch 2/30
1050/1050 [==============================] - 41s 39ms/step - loss: 0.0696 - accuracy: 0.9818 - val_loss: 0.0601 - val_accuracy: 0.9851
Epoch 3/30
1050/1050 [==============================] - 41s 39ms/step - loss: 0.0599 - accuracy: 0.9835 - val_loss: 0.0490 - val_accuracy: 0.9871
Epoch 4/30
1050/1050 [==============================] - 41s 39ms/step - loss: 0.0562 - accuracy: 0.9846 - val_loss: 0.0375 - val_accuracy: 0.9898
Epoch 5/30
1050/1050 [==============================] - 41s 39ms/step - loss: 0.0495 - accuracy: 0.9863 - val_loss: 0.0387 - val_accuracy: 0.9896
Epoch 6/30
1050/1050 [==============================] - 42s 40ms/step - loss: 0.0509 - accuracy: 0.9862 - val_loss: 0.0385 - val_accuracy: 0.9898
Epoch 7/30
1050/1050 [==============================] - 43s 41ms/step - loss: 0.0437 - accuracy: 0.9887 - val_loss: 0.0365 -

## Submission 

In [33]:
aug_ans = pd.DataFrame({
    'ImageId': 1 + np.arange(len(X_test)),
    'Label': model.predict_classes(X_test)
})

aug_ans.to_csv('digit_prediction_augmentation.csv', index = False)